<a href="https://colab.research.google.com/github/simodepth/Autosuggest-Keywords/blob/main/Keyword_Research_and_Clustering_with_Google_Suggest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Make a Keyword Research in 5 minutes using Google Suggest**

---


This Keyword Script will quickly generate keyword ideas from Google Suggest and cluster them using NLP transformers

## Step 1: Change Settings

Fill in the field lang_code with your language code (e.g. en, fr, es, nl) and enter up to 5 seed keywords you want to use.



In [ ]:
#language code and keywords
lang_code="en"#@param {type:"string"}
keyword1="digital agency" #@param {type:"string"}
keyword2="marketing" #@param {type:"string"}
keyword3="leeds" #@param {type:"string"}
keyword4="" #@param {type:"string"}
keyword5="" #@param {type:"string"}
numclusters=10 #@param
numsubclusters=5 #@param

## Step 2: Run The Code

In [ ]:
#generate keyword list
keywords=[keyword1,keyword2,keyword3,keyword4,keyword5]
keywordlist = list(filter(None, keywords))
keywordlist

['digital agency', 'marketing', 'leeds']

### Importeer modules

In [ ]:
!pip install sentence_transformers

import pandas as pd
import numpy as np
import requests
import json
import time
import string
import nltk
nltk.download('punkt')
!pip install stop_words
from stop_words import get_stop_words
from google.colab import files
%load_ext google.colab.data_table
from collections import Counter
from json import loads
import re
from nltk.cluster import KMeansClusterer, euclidean_distance
from scipy.spatial import distance_matrix
from sentence_transformers import SentenceTransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 2.5 MB/s 
     |████████████████████████████████| 4.2 MB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.5 MB/s 
     |████████████████████████████████| 86 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=e356397ba90233895869a99be790ab2542d339e11f3b4f4de6ef1a1b45369c38
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipp

In [ ]:
#Make a list of letters to use for Google Suggest
letterlist=[""]
letterlist=letterlist+list(string.ascii_lowercase)

In [ ]:
#Google Suggest for each combination of keyword and letter
keywordsuggestions=[]
for keyword in keywordlist: 
  for letter in letterlist :
    URL="http://suggestqueries.google.com/complete/search?client=firefox&hl="+str(lang_code)+"&q="+keyword+" "+letter
    headers = {'User-agent':'Mozilla/5.0'} 
    response = requests.get(URL, headers=headers) 
    result = json.loads(response.content.decode('utf-8'))
    keywordsuggest=[keyword,letter] 
    for word in result[1]:
      if(word!=keyword):
        keywordsuggest.append(word)
    time.sleep(1)
    keywordsuggestions.append(keywordsuggest)
#crearte a dataframe from this list
keywordsuggestions_df = pd.DataFrame(keywordsuggestions)

In [ ]:
#Rename columns of dataframe
columnnames=["Keyword","Letter"]
for i in range(1,len(keywordsuggestions_df.columns)-1):
  columnnames.append("Suggestion"+str(i))
keywordsuggestions_df.columns=columnnames

In [ ]:
#Make a list of all suggestions
allkeywords = keywordlist
for i in range(1,len(keywordsuggestions_df.columns)-1):
  suggestlist = keywordsuggestions_df["Suggestion"+str(i)].values.tolist()
  for suggestion in suggestlist:
    allkeywords.append(suggestion)

In [ ]:
model = SentenceTransformer('clips/mfaq')

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
embeddings = model.encode(keywordlist)
data=[]
i=0
for keyword in allkeywords:
    try:
      data.append([keyword,embeddings[i]])
      i=i+1
    except:
      print("error")
keywordsdf = pd.DataFrame(data,columns=["Keyword","Emb"])
embeddinglist = keywordsdf["Emb"].values.tolist()

In [ ]:
def clustering_question(data,NUM_CLUSTERS = 7):

    sentences = data['Keyword']

    X = np.array(data['Emb'].tolist())

    kclusterer = KMeansClusterer(
        NUM_CLUSTERS, distance=nltk.cluster.util.cosine_distance,
        repeats=30,avoid_empty_clusters=True)

    assigned_clusters = kclusterer.cluster(X, assign_clusters=True)

    data['cluster'] = pd.Series(assigned_clusters, index=data.index)
    data['centroid'] = data['cluster'].apply(lambda x: kclusterer.means()[x])

    return data, assigned_clusters
def distance_from_centroid(row):
    # type of emb and centroid is different, hence using tolist below
    return distance_matrix([row['Emb']], [row['centroid'].tolist()])[0][0]


In [ ]:
clustervalues = clustering_question(keywordsdf,numclusters)
df = clustervalues[0]

df['distance_from_centroid'] = df.apply(distance_from_centroid, axis=1)
distance = df["distance_from_centroid"].sum()

In [ ]:
clusterdfs=[]
for i in range(0,numclusters):

    clusterdf = df[df["cluster"]==i]
    #subcluster trainen
    try:
      clustervalues = clustering_question(clusterdf,numsubclusters)
      #print(i)
      # print(clustervalues[0])
      clusterdfs.append(clustervalues[0])
    except:
      print("small subcluster")
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#zoek subcluster
def find_subcluster(keyword,cluster):
  try:
    #print(keyword,cluster)
    subclusterdf = clusterdfs[cluster]
    #print(subclusterdf.head())
    #print(subclusterdf[subclusterdf["Keyword"]==keyword]["cluster"].values)
    return subclusterdf[subclusterdf["Keyword"]==keyword]["cluster"].values.tolist()[0]
  except:
    return 9

In [ ]:
df["subcluster"]=df.apply(lambda x:find_subcluster(x["Keyword"],x["cluster"]),axis=1)
#df.head()

In [ ]:
df["cluster-subcluster"]=df.apply(lambda x:int(str(x["cluster"])+str(x["subcluster"])),axis=1)

In [ ]:
clusternames=[]
maxcluster = df["cluster"].max()
for i in range(0,maxcluster+1):
    keywords = df[df["cluster"]==i]["Keyword"].values.tolist()
    words=[]
    for keyword in keywords:
        for word in keyword.split(' '):
            word = word.lower()
            word = word.replace('?','')
            words.append(word)
    most_common_words= [word for word, word_count in Counter(words).most_common(5)]
    clustername = most_common_words[0]+" "+most_common_words[1]+" "+most_common_words[2]+" "+most_common_words[3]+" "+most_common_words[4]
    clusternames.append(clustername)
clusternames

['marketing examples vs your definition',
 'marketing and manager operations lead',
 'leeds united weather kit house',
 'digital agency of evero the',
 'leeds school business outlet university',
 'leeds vs england united news',
 'digital agency template website design',
 'marketing salary jobs california texas',
 'digital agency jakarta york california',
 'agency digital marketing in vs']

In [ ]:
subclusternames=[]
maxsubcluster = df["cluster-subcluster"].max()
for i in range(0,maxsubcluster+1):
    try:
        keywords = df[df["cluster-subcluster"]==i]["Keyword"].values.tolist()
        words=[]
        for keyword in keywords:
            for word in keyword.split(' '):
                word = word.lower()
                word = word.replace('?','')
                words.append(word)
        most_common_words= [word for word, word_count in Counter(words).most_common(5)]
        subclustername = most_common_words[0]+" "+most_common_words[1]+" "+most_common_words[2]+" "+most_common_words[3]+" "+most_common_words[4]
    
        subclusternames.append(subclustername)
    except:
        subclusternames.append("")
subclusternames

['marketing definition goals objectives examples',
 'marketing news websites books gifts',
 'marketing vs your on strategy',
 'marketing plan template budget kit',
 'marketing questions examples for keywords',
 '',
 '',
 '',
 '',
 '',
 'marketing and research fx zone',
 'marketing operations ethics information management',
 'marketing lead skills hub near',
 'marketing internships university degree resume',
 'marketing manager consultant operations analyst',
 '',
 '',
 '',
 '',
 '',
 'leeds united drive in vs',
 'leeds public house injuries and',
 'leeds weather news illuminate injury',
 'leeds infirmary xg nd pcna',
 'leeds kit dog backpack eye',
 '',
 '',
 '',
 '',
 '',
 'digital agency evero groove usa',
 'digital agency of the year',
 'digital agency evero app login',
 'digital agency nyc united inc',
 'digital agency manager labs project',
 '',
 '',
 '',
 '',
 '',
 'leeds group youth athletic association',
 'leeds manager coach united transfer',
 'leeds outlet park for mall',
 'le

In [ ]:
df["clustername"]=df.apply(lambda x: clusternames[x["cluster"]],axis=1)
df["subclustername"]=df.apply(lambda x: subclusternames[x["cluster-subcluster"]],axis=1)
#sort by cluster-subcluster
df = df.sort_values(by=['cluster-subcluster'])
df[["Keyword","cluster","subcluster","clustername","subclustername","cluster-subcluster"]].to_csv('clustered-keywords.csv')
df[["Keyword","cluster","subcluster","clustername","subclustername","cluster-subcluster"]]

,Keyword,cluster,subcluster,clustername,subclustername,cluster-subcluster
451,marketing plan definition,0,0,marketing examples vs your definition,marketing definition goals objectives examples,0
604,marketing goals and objectives,0,0,marketing examples vs your definition,marketing definition goals objectives examples,0
291,marketing research definition,0,0,marketing examples vs your definition,marketing definition goals objectives examples,0
775,marketing principles,0,0,marketing examples vs your definition,marketing definition goals objectives examples,0
776,marketing que es,0,0,marketing examples vs your definition,marketing definition goals objectives examples,0
...,...,...,...,...,...,...
515,digital marketing agency zambia,9,3,agency digital marketing in vs,digital marketing agency in york,93
352,digital marketing agency york,9,3,agency digital marketing in vs,digital marketing agency in york,93
660,digital agency in mumbai,9,4,agency digital marketing in vs,digital agency in atlanta karachi,94
417,digital agency in karachi,9,4,agency digital marketing in vs,digital agency in atlanta karachi,94


In [ ]:
from google.colab import files
files.download('clustered-keywords.csv') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>